# setup

> the `export as pdf` feature of the jupyter doesn't support chinese characters, so I have to use english to write the markdown.

Please refer to the `README.md` for the chinese version of the document and make sense of the setup.

In [ ]:
from math import sqrt
import numpy as np
from scipy import interpolate as itp
from utils.itp import lagrange as lag  # lagrange interpolation is implemented in the `utils` package

In [ ]:
# define the exact function
def func_y(x: float) -> float:
  return sqrt(x)


# generate the sample points, use list for easy to split
lt_X = [0.0, 1.0, 4.0, 9.0, 16.0]
lt_y = [func_y(x) for x in lt_X]

# The data below four points are meaningless for the interpolation
# For the deviation is too large for the exact value
# So i won't handle them in the following process

lts_X = [lt_X, lt_X[1:], lt_X[:1] + lt_X[2:], lt_X[:2] + lt_X[3:], lt_X[:3] + lt_X[4:], lt_X[:4]]

lts_y = [lt_y, lt_y[1:], lt_y[:1] + lt_y[2:], lt_y[:2] + lt_y[3:], lt_y[:3] + lt_y[4:], lt_y[:4]]

# turn the list into numpy array
X_origins = [np.array(lt) for lt in lts_X]
y_origins = [np.array(lt) for lt in lts_y]

# generate the target point
X_test = np.array([5.0])
y_exact = np.array([func_y(5.0)])
print(f'The exact value of y at x = 5 is {y_exact[0]}')


# define the deviation function
def deviation(_y1: np.ndarray, _y2: np.ndarray) -> np.ndarray:
  return np.abs(_y1 - _y2)

In [ ]:
# lagrange interpolation, which returns an value
for X_origin, y_origin in zip(X_origins, y_origins):
  y_lag = lag(X_origin, y_origin, X_test)
  y_lag_dev = deviation(y_lag, y_exact)  # type: ignore
  print(f'{y_lag[0]:.4f}, {y_lag_dev[0]:.4f}')  # type: ignore

In [ ]:
# cubic spline interpolation, which returns a function
for X_origin, y_origin in zip(X_origins, y_origins):
  f_spline = itp.interp1d(X_origin, y_origin, kind='cubic')
  y_spline = f_spline(X_test)
  y_spline_dev = deviation(y_spline, y_exact)
  print(f'{y_spline[0]:.4f}, {y_spline_dev[0]:.4f}')

In [ ]:
# segmented linear interpolation, which returns a function
for X_origin, y_origin in zip(X_origins, y_origins):
  f_linear = itp.interp1d(X_origin, y_origin, kind='linear')
  y_linear = f_linear(X_test)
  y_linear_dev = deviation(y_linear, y_exact)
  print(f'{y_linear[0]:.4f}, {y_linear_dev[0]:.4f}')

# result

|      method      |   sample    | deviation |
|:----------------:|:-----------:|:---------:|
|     lagrange     | no excluded |  0.1567   |
|     lagrange     | exclude P1  |  0.0179   |
|     lagrange     | exclude P2  |  0.0616   |
|     lagrange     | exclude P3  |  0.7163   |
|     lagrange     | exclude P4  |  0.3750   |
|     lagrange     | exclude P5  |  0.2361   |
|                  |             |           |
|   cubic spline   | no excluded |  0.0728   |
|   cubic spline   | exclude P1  |  0.0179   |
|   cubic spline   | exclude P2  |  0.0616   |
|   cubic spline   | exclude P3  |  0.7163   |
|   cubic spline   | exclude P4  |  0.3750   |
|   cubic spline   | exclude P5  |  0.2361   |
|                  |             |           |
| segmented linear | no excluded |  0.0361   |
| segmented linear | exclude P1  |  0.0361   |
| segmented linear | exclude P2  |  0.0361   |
| segmented linear | exclude P3  |  0.2361   |
| segmented linear | exclude P4  |  0.0694   |
| segmented linear | exclude P5  |  0.0361   |

- Lagrangian interpolation is strongly influenced by the outburst value.
- Cubic spline interpolation is also influenced by the outburst value, but not as much as the Lagrangian interpolation.
- Segmented linear interpolation is only affected by nearby values. When the interval is expanded, the deviation changes significantly.

Of the three methods, segmented linear interpolation performs best while the lagrange interpolation performs worst.